In [1]:
import os
import pandapower as pp
import polars as pl
from polars import col as c

from pipelines.dig_a_plan_complete_master import DigAPlan
from data_display.grid_plotting import plot_grid_from_pandapower
from data_display.output_processing import compare_dig_a_plan_with_pandapower
from data_connector import change_schema_to_dig_a_plan_schema, duckdb_to_changes_schema
from twindigrid_changes.schema import ChangesSchema
from networkx_function import generate_nx_edge, get_connected_edges_data, generate_bfs_tree_with_edge_data, get_all_edge_data
import networkx as nx

import matplotlib.pyplot as plt

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"


In [2]:
change_schema: ChangesSchema = duckdb_to_changes_schema(file_path=".cache/input_data/boisy_grid.db")
# 2) build the *original* schema once

base_grid_data = change_schema_to_dig_a_plan_schema(change_schema = change_schema)

nx_graph = nx.Graph()
_ = base_grid_data["edge_data"].with_columns(
    pl.struct(pl.all()).pipe(generate_nx_edge, nx_graph=nx_graph)
)

max_n_trafo = 1.08
edge_id  = get_connected_edges_data(nx_graph = nx_graph).filter(c("graph_id")!= 0)["edge_id"].to_list()
node_id = get_connected_edges_data(nx_graph = nx_graph).filter(c("graph_id")!= 0)\
    .unpivot(on=["u_of_edge", "v_of_edge"])["value"].to_list()
base_grid_data["edge_data"] = base_grid_data["edge_data"].filter(~c("edge_id").is_in(edge_id))
base_grid_data["node_data"] = base_grid_data["node_data"].filter(~c("node_id").is_in(node_id))
base_grid_data["edge_data"] = base_grid_data["edge_data"].with_columns(
    pl.when(c("b_pu") < 1e-6).then(pl.lit(0)).otherwise(c("b_pu")).alias("b_pu"),
    pl.when(c("n_transfo") > max_n_trafo).then(pl.lit(max_n_trafo)).otherwise(c("n_transfo")).alias("n_transfo")
    )


Read and validate tables from boisy_grid.db file: 100%|██████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 341.98it/s]


In [3]:

dig_a_plan: DigAPlan = DigAPlan(
    verbose= False, big_m = 1e4, penalty_cost = 1e1, convergence_threshold = 1e-6
)

dig_a_plan.add_grid_data(**base_grid_data)
dig_a_plan.solve_models_pipeline(max_iters = 200)

Master obj: 0, Slave obj: 0 and Gap: 1e6:   0%|          | 0/200 [00:00<?, ?it/s]


ValueError: Cannot load a SolverResults object with bad status: error

In [ ]:
dig_a_plan.master_obj

-1e-08

In [ ]:
dig_a_plan.master_model_instance.objective()

-1e-08

In [ ]:
dig_a_plan.master_model_instance.theta.extract_values()

{None: -1e-08}

In [ ]:
dig_a_plan.master_model_instance.optimality_cut.pprint()

optimality_cut : Size=1, Index={1}, Active=True
    Key : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [ ]:
master_ds = dig_a_plan.master_model_instance.d.extract_values() # type: ignore
dig_a_plan.slave_model_instance.master_d.store_values(master_ds) # type: ignore
results = dig_a_plan.slave_solver.solve(dig_a_plan.slave_model_instance, tee=False)
dig_a_plan.slave_model_instance.objective() # type: ignore
            

0.02824380513548836

In [ ]:
dig_a_plan.master_model_instance.optimality_cut.pprint()
                                    

optimality_cut : Size=1, Index={1}, Active=True
    Key : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [ ]:
from pyomo_utility import extract_optimization_results


master_d = extract_optimization_results(dig_a_plan.slave_model_instance, "master_d").with_columns(
    c("LC").cast(pl.List(pl.Utf8)).list.join(","),
    c("LC").list.get(0).alias("l"),
)

d = extract_optimization_results(dig_a_plan.master_model_instance, "d").with_columns(
    c("LC").cast(pl.List(pl.Utf8)).list.join(","),
    c("d").cast(pl.Int32)
)
unmatch = master_d.join(d, on="LC", how="left").filter(c("master_d") != c("d")).join(dig_a_plan.marginal_cost, on="LC", how="left")
print(unmatch.to_pandas().to_string())

Empty DataFrame
Columns: [LC, master_d, l, d, marginal_cost]
Index: []


In [ ]:
unmatch

LC,master_d,l,d,marginal_cost
str,f64,i64,i32,f64


In [ ]:
dig_a_plan.master_model_instance.d.extract_values()

{(0, 36, 0): 0.0,
 (1, 1, 108): 0.0,
 (2, 4, 19): 1.0,
 (3, 27, 5): 0.0,
 (4, 80, 15): 0.0,
 (5, 16, 102): 0.0,
 (6, 20, 70): 1.0,
 (7, 81, 23): 0.0,
 (8, 8, 71): 0.0,
 (9, 9, 82): 0.0,
 (10, 10, 83): 0.0,
 (11, 28, 62): 0.0,
 (12, 84, 29): 0.0,
 (13, 32, 68): 0.0,
 (14, 33, 114): 1.0,
 (15, 85, 37): 0.0,
 (16, 54, 40): 0.0,
 (17, 42, 55): 0.0,
 (18, 103, 43): 0.0,
 (19, 109, 60): 0.0,
 (20, 63, 112): 0.0,
 (21, 115, 72): 1.0,
 (22, 118, 75): 1.0,
 (23, 91, 119): 0.0,
 (24, 105, 98): 0.0,
 (25, 564, 965): 0.0,
 (26, 564, 874): 1.0,
 (27, 874, 163): 1.0,
 (28, 595, 564): 1.0,
 (29, 489, 874): 1.0,
 (30, 671, 163): 1.0,
 (31, 666, 652): 0.0,
 (32, 416, 652): 1.0,
 (33, 587, 652): 1.0,
 (34, 391, 666): 1.0,
 (35, 666, 485): 1.0,
 (36, 485, 279): 1.0,
 (37, 603, 279): 1.0,
 (38, 1005, 603): 1.0,
 (39, 163, 774): 0.0,
 (40, 774, 615): 0.0,
 (41, 493, 615): 1.0,
 (42, 768, 615): 1.0,
 (43, 926, 774): 1.0,
 (44, 603, 238): 0.0,
 (45, 291, 485): 1.0,
 (46, 784, 334): 1.0,
 (47, 784, 873): 0.0,

In [ ]:
dig_a_plan.master_model_instance.theta.extract_values()

{None: -1e-08}

In [ ]:
dig_a_plan.extract_switch_status().filter(c("open") != c("normal_open"))

eq_fk,normal_open,open
str,bool,bool
"""d9255bb0-767a-5eec-81ff-0a3df1…",true,false
"""922efaff-4656-579c-8b53-e525e2…",true,false
"""ed143c3e-82cc-5976-aa7b-e81e89…",true,false
"""0e05522f-cbe2-5b34-ae55-9a9204…",true,false
"""9bacd1d0-7b6f-591e-b3c3-369f5b…",true,false
…,…,…
"""983071ce-9611-51a4-b868-ca8c5e…",false,true
"""c7764c22-c0dc-558b-89f4-6cc58b…",true,false
"""4d29ba1b-6de6-594d-b68d-f578c6…",false,true


In [ ]:
dig_a_plan.extract_switch_status()

eq_fk,normal_open,open
str,bool,bool
"""8dbec791-0228-5627-a843-2bf35f…",false,false
"""69a27e56-783b-5035-b29c-d5504a…",false,false
"""19da717d-66b8-5214-9e55-10ec53…",false,false
"""bd77cbbd-2689-5ae3-a6c9-62f159…",false,false
"""3701ef23-c779-5179-b360-c38198…",false,false
…,…,…
"""0ad85241-4389-5b70-a9ba-556bee…",false,false
"""3ca0edea-88fb-56c6-a607-cf81ce…",true,true
"""9000acee-5204-502e-ba00-9692fe…",false,false
